In [1]:
import pandas as pd
import gdown
import sqlite3 as sql3

In [2]:
url='https://drive.google.com/file/d/1LCpP0FD-JFwJ0V0k2wbi15Q7s3goGZfa/view?usp=drive_link'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
gdown.download(url, 'chinook.db', quiet=False)
connection = sql3.connect('chinook.db')

Downloading...
From: https://drive.google.com/uc?id=1LCpP0FD-JFwJ0V0k2wbi15Q7s3goGZfa
To: c:\Users\mail\OneDrive\Documents\my_python\zooming_up\SQL\chinook.db
100%|██████████| 885k/885k [00:00<00:00, 3.32MB/s]


Агрегат по колонке всей таблицы схлопывает таблицу

In [120]:
query = """
        WITH cte AS
        (SELECT  invoice_items.quantity as qty, tracks.name as track, albums.title as album
        FROM    invoice_items
                LEFT JOIN tracks
                ON tracks.trackid = invoice_items.trackid
                LEFt JOIN albums
                ON albums.AlbumId= tracks.AlbumId)

        SELECT *, COUNT(1) as cnt
        FROM cte
        LIMIT 3
        """
pd.read_sql_query(query, connection)

,qty,track,album,cnt
0,1,Balls to the Wall,Balls to the Wall,2240


Агрегат, вынесенный в подзапрос, передается как константа в каждую запись

In [123]:
query = """
        WITH cte AS
        (SELECT  invoice_items.quantity as qty, tracks.name as track, albums.title as album
        FROM    invoice_items
                LEFT JOIN tracks
                ON tracks.trackid = invoice_items.trackid
                LEFt JOIN albums
                ON albums.AlbumId= tracks.AlbumId)

        SELECT *, 
                (SELECT count(qty)
                FROM cte) as avg
        FROM cte
        LIMIT 3
        """
pd.read_sql_query(query, connection)

,qty,track,album,avg
0,1,Balls to the Wall,Balls to the Wall,2240
1,1,Restless and Wild,Restless and Wild,2240
2,1,Put The Finger On You,For Those About To Rock We Salute You,2240


OVER() - агрегат над всей таблицей - так же передается как константа в каждую запись

In [133]:
query = """
        WITH cte AS
        (SELECT  invoice_items.quantity as qty, tracks.name as track, albums.title as album
        FROM    invoice_items
                LEFT JOIN tracks
                ON tracks.trackid = invoice_items.trackid
                LEFt JOIN albums
                ON albums.AlbumId= tracks.AlbumId)

        SELECT *, COUNT(1)
        OVER () as cnt
        FROM cte
        LIMIT 3
        """
pd.read_sql_query(query, connection)

,qty,track,album,cnt
0,1,Balls to the Wall,Balls to the Wall,2240
1,1,Restless and Wild,Restless and Wild,2240
2,1,Put The Finger On You,For Those About To Rock We Salute You,2240


Агрегация по группировке в столбце, схлопывает группы в запись и добавляет поле со значениями новых записей (схлопнутых групп)

In [129]:
query = """
        WITH cte AS
        (SELECT  invoice_items.quantity as qty, tracks.name as track, albums.title as album
        FROM    invoice_items
                LEFT JOIN tracks
                ON tracks.trackid = invoice_items.trackid
                LEFt JOIN albums
                ON albums.AlbumId= tracks.AlbumId)

        SELECT *, COUNT(1) as cnt
        FROM cte
        GROUP BY album
        ORDER BY cnt DESC
        LIMIT 3
                """
pd.read_sql_query(query, connection)

,qty,track,album,cnt
0,1,"Bye, Bye Brasil",Minha Historia,27
1,1,Are You Gonna Go My Way,Greatest Hits,26
2,1,Tears In Heaven,Unplugged,25


OVER (PARTITION BY) считает тоже самое что и GROUP BY, но не схлопывает записи в группу, а вписывает в каждую запись агрегат для группы

In [132]:
query = """
        WITH cte AS
        (SELECT  invoice_items.quantity as qty, tracks.name as track, albums.title as album
        FROM    invoice_items
                LEFT JOIN tracks
                ON tracks.trackid = invoice_items.trackid
                LEFt JOIN albums
                ON albums.AlbumId= tracks.AlbumId)

        SELECT *, COUNT(1)
        OVER (PARTITION BY album) as cnt
        FROM cte
        ORDER BY cnt DESC
        LIMIT 3
        """
pd.read_sql_query(query, connection)

,qty,track,album,cnt
0,1,"Bye, Bye Brasil",Minha Historia,27
1,1,Atras Da Porta,Minha Historia,27
2,1,Tatuagem,Minha Historia,27


Добавялем сортировку по названию трека и без указания range по умолчанию считается RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
то есть числотреков считается не все для всего альбома, а от начала альбома до конкретной композиции, то есть кумулятивный счетчик
и для композиции с самой далекой от "а" первой буквой получается максимальный cnt. И так для каждого аольбома

In [146]:
query = """
        WITH cte AS
        (SELECT  invoice_items.quantity as qty, tracks.name as track, albums.title as album
        FROM    invoice_items
                LEFT JOIN tracks
                ON tracks.trackid = invoice_items.trackid
                LEFt JOIN albums
                ON albums.AlbumId= tracks.AlbumId)

        SELECT *, COUNT(1)
        OVER (PARTITION BY album
              ORDER BY track ) as cnt
        FROM cte
        ORDER BY cnt DESC
        LIMIT 5
    
        """
pd.read_sql_query(query, connection)

,qty,track,album,cnt
0,1,Voce Nao Entende Nada - Cotidiano,Minha Historia,27
1,1,You're Gonna Break My Hart Again,Greatest Hits,26
2,1,Vai Passar,Minha Historia,26
3,1,Wanted Dread And Alive,Greatest Hits,25
4,1,Um Indio,Minha Historia,25


Если же задать размеры диапазона RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING, то есть всю партицию, то и cnt для каждой строки будет считаться по всей партиции

In [149]:
query = """
        WITH cte AS
        (SELECT  invoice_items.quantity as qty, tracks.name as track, albums.title as album
        FROM    invoice_items
                LEFT JOIN tracks
                ON tracks.trackid = invoice_items.trackid
                LEFt JOIN albums
                ON albums.AlbumId= tracks.AlbumId)

        SELECT *, COUNT(1)
        OVER (PARTITION BY album
              ORDER BY track
              RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as cnt
        FROM cte
        ORDER BY cnt DESC
        LIMIT 3
    
        """
pd.read_sql_query(query, connection)

,qty,track,album,cnt
0,1,A Banda,Minha Historia,27
1,1,Apesar De Você,Minha Historia,27
2,1,Atras Da Porta,Minha Historia,27


RANGE - определяется значением, которое формирует партицию. Все треки альбома входят в один range, и потому для всех считается одно и тоже значение

In [155]:
query = """
        WITH cte AS
        (SELECT  invoice_items.quantity as qty, tracks.name as track, albums.title as album
        FROM    invoice_items
                LEFT JOIN tracks
                ON tracks.trackid = invoice_items.trackid
                LEFt JOIN albums
                ON albums.AlbumId= tracks.AlbumId)

        SELECT *, COUNT(1)
        OVER (PARTITION BY album
              ORDER BY album
              RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as cnt
        FROM cte
        ORDER BY cnt DESC
        LIMIT 10
    
        """
pd.read_sql_query(query, connection)

,qty,track,album,cnt
0,1,"Bye, Bye Brasil",Minha Historia,27
1,1,Atras Da Porta,Minha Historia,27
2,1,Tatuagem,Minha Historia,27
3,1,Morena De Angola,Minha Historia,27
4,1,A Banda,Minha Historia,27
5,1,Com Açúcar E Com Afeto,Minha Historia,27
6,1,Meu Caro Amigo,Minha Historia,27
7,1,Trocando Em Miúdos,Minha Historia,27
8,1,Gota D'água,Minha Historia,27
9,1,Podres Poderes,Minha Historia,27


ROWS определяет строки до или после текущей, и для каждого трека альбома внутри партиции считается разный результат

In [157]:
query = """
        WITH cte AS
        (SELECT  invoice_items.quantity as qty, tracks.name as track, albums.title as album
        FROM    invoice_items
                LEFT JOIN tracks
                ON tracks.trackid = invoice_items.trackid
                LEFt JOIN albums
                ON albums.AlbumId= tracks.AlbumId)

        SELECT *, COUNT(1)
        OVER (PARTITION BY album
              ORDER BY album
              ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as cnt
        FROM cte
        ORDER BY cnt DESC
        LIMIT 10
    
        """
pd.read_sql_query(query, connection)

,qty,track,album,cnt
0,1,Menino Do Rio,Minha Historia,27
1,1,Sweet Lady Luck,Greatest Hits,26
2,1,Geni E O Zepelim,Minha Historia,26
3,1,Looking For Love,Greatest Hits,25
4,1,Apesar De Você,Minha Historia,25
5,1,Toda Menina Baiana,Unplugged,25
6,1,If You Don't Know Me By Now,Greatest Hits,24
7,1,Samba De Orly,Minha Historia,24
8,1,Tempo Rei,Unplugged,24
9,1,Pick Myself Up,Greatest Hits,23


Агрегатные оконные функции SUM COUNT MIN MAX AVG

In [181]:
query = """

        SELECT  date(InvoiceDate) as date,
                weeknum,
                Total, 
                COUNT(Total) OVER (PARTITION BY weeknum ORDER BY InvoiceDate RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as cnt,
                MIN(Total) OVER (PARTITION BY weeknum ORDER BY InvoiceDate RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as MIN,
                MAX(Total) OVER (PARTITION BY weeknum ORDER BY InvoiceDate RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as MAX,
                SUM(Total) OVER (PARTITION BY weeknum ORDER BY InvoiceDate RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as SUM,
                AVG(Total) OVER (PARTITION BY weeknum ORDER BY InvoiceDate RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as AVG
        FROM 
            (SELECT InvoiceDate,
                    strftime('%W', InvoiceDate) as weeknum,
                    Total
            FROM invoices)
        -- ORDER BY cnt DESC
        LIMIT 10
    
        """
pd.read_sql_query(query, connection)

,date,weeknum,Total,cnt,MIN,MAX,SUM,AVG
0,2009-01-01,00,1.98,6,0.99,13.86,35.64,5.940000
1,2009-01-02,00,3.96,6,0.99,13.86,35.64,5.940000
2,2009-01-03,00,5.94,6,0.99,13.86,35.64,5.940000
3,2011-01-02,00,0.99,6,0.99,13.86,35.64,5.940000
4,2012-01-01,00,13.86,6,0.99,13.86,35.64,5.940000
5,2013-01-02,00,8.91,6,0.99,13.86,35.64,5.940000
6,2009-01-06,01,8.91,7,1.98,13.86,51.49,7.355714
7,2009-01-11,01,13.86,7,1.98,13.86,51.49,7.355714
8,2010-01-08,01,1.98,7,1.98,13.86,51.49,7.355714
9,2010-01-08,01,1.98,7,1.98,13.86,51.49,7.355714


Ранговые функции: ROW_NUMBER, RANK, DENSE_RANK

In [185]:
query = """
        SELECT  date(InvoiceDate) as date,
                weeknum,
                Total, 
                ROW_NUMBER() OVER (PARTITION BY weeknum ORDER BY total RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as rnum,
                RANK() OVER (PARTITION BY weeknum ORDER BY total RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as rank,
                DENSE_RANK() OVER (PARTITION BY weeknum ORDER BY total RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as drank
                FROM 
            (SELECT InvoiceDate,
                    strftime('%W', InvoiceDate) as weeknum,
                    Total
            FROM invoices)
        -- ORDER BY cnt DESC
        LIMIT 10
    
        """
pd.read_sql_query(query, connection)

,date,weeknum,Total,rnum,rank,drank
0,2011-01-02,00,0.99,1,1,1
1,2009-01-01,00,1.98,2,2,2
2,2009-01-02,00,3.96,3,3,3
3,2009-01-03,00,5.94,4,4,4
4,2013-01-02,00,8.91,5,5,5
5,2012-01-01,00,13.86,6,6,6
6,2010-01-08,01,1.98,1,1,1
7,2010-01-08,01,1.98,2,1,1
8,2010-01-09,01,3.96,3,3,2
9,2010-01-10,01,6.94,4,4,3


Функции смещения : LAG, LEAD, FIRST_VALUE, LAST_VALUE

In [190]:
query = """
        SELECT  date(InvoiceDate) as date,
                weeknum,
                Total, 
                LAG(total) OVER (PARTITION BY weeknum ORDER BY total RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as lag,
                LEAD(total) OVER (PARTITION BY weeknum ORDER BY total RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as lead,
                FIRST_VALUE(total) OVER (PARTITION BY weeknum ORDER BY total RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as first,
                LAST_VALUE(total) OVER (PARTITION BY weeknum ORDER BY total RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as last
                FROM 
            (SELECT InvoiceDate,
                    strftime('%W', InvoiceDate) as weeknum,
                    Total
            FROM invoices)
        -- ORDER BY cnt DESC
        LIMIT 10
    
        """
pd.read_sql_query(query, connection)

,date,weeknum,Total,lag,lead,first,last
0,2011-01-02,00,0.99,NaN,1.98,0.99,13.86
1,2009-01-01,00,1.98,0.99,3.96,0.99,13.86
2,2009-01-02,00,3.96,1.98,5.94,0.99,13.86
3,2009-01-03,00,5.94,3.96,8.91,0.99,13.86
4,2013-01-02,00,8.91,5.94,13.86,0.99,13.86
5,2012-01-01,00,13.86,8.91,NaN,0.99,13.86
6,2010-01-08,01,1.98,NaN,1.98,1.98,13.86
7,2010-01-08,01,1.98,1.98,3.96,1.98,13.86
8,2010-01-09,01,3.96,1.98,6.94,1.98,13.86
9,2010-01-10,01,6.94,3.96,8.91,1.98,13.86
